<a href="https://colab.research.google.com/github/sonahyoung/class2022Spring/blob/%EA%B8%B0%EB%A7%90%EB%B2%94%EC%9C%84/huggingface_gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio
import gradio as gr

In [ ]:
gr.Interface.load("huggingface/gpt2").launch()  #이런 식으로 바로 되는 모델이 있고 안되는 게 있음

### [Image classification](https://huggingface.co/tasks/image-classification)


사이트 들어가서 image classification 정의도 알아놓기, 시험문제에 이건 안다고 가정하고 낼거임

e.g. https://huggingface.co/google/vit-base-patch16-224 \
How to use

에러메세지가 나오면 그거에 맞춰서 다시 해줘야함

In [ ]:
!pip install transformers

In [5]:
from transformers import ViTFeatureExtractor, ViTForImageClassification
from PIL import Image
import requests

url = 'http://images.cocodataset.org/val2017/000000039769.jpg'  #이미지가 있어야지 classification이 되는거니까 이미지를 불러와야함
image = Image.open(requests.get(url, stream=True).raw)  #url을 불러와서 이미지를 오픈해와서, image가 숫자데이터로 저장이 되어있다. image 변수가 그림이라고 생각하면 됨.

feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')
#pretrained는 행렬임 (데이터로부터 잘 학습이 된 행렬) - Feature Extraction하는 모델, extracted된 정보를 가지고 classfication하는 모델 (변수 이름은 내 마음대로 정할 수 있지만 보고 추측하는 거임) 
#color image: 3층 정보 (R, G, B) / 흑백 이미지: 1층 정보 

inputs = feature_extractor(images=image, return_tensors="pt")  #숫자들 속에서 더 압축적으로 쓸 수 있는 정보로 뽑아내는 것
#ex. 사람에 대한 정보를 뽑아낸다고 치면 무한대의 정보가 있지만, 그걸 feature 형태로 뽑아낸다고 하면, 눈썹이 어떻게 생겼고, 눈은 어떻게 생겼고, 남자고 여자고 이런거를 뽑아내는거임. object를 기술할 수 있는 핵심을 feature라고 생각하면 됨
#그렇게 feature된 정보는 original raw 데이터보다는 개수가 더 작을거임
outputs = model(**inputs)  #output은 classified된 정보가 나옴
logits = outputs.logits
# model predicts one of the 1000 ImageNet classes    #1000개의 class가 원래 들어가 있음. 그것중에 하나를 predict해내는 거임, 1000개의 숫자가 뽑아져나옴. 그거에 대한 확률이 다 나와서 다 더하면 1이 됨. 
predicted_class_idx = logits.argmax(-1).item()
print("Predicted class:", model.config.id2label[predicted_class_idx])  #그 중에서 확률이 제일 큰 걸 보여주라고 함 거임.

Predicted class: Egyptian cat


이걸 이제 gradio화를 해야함. function 만들고 input, output 만들어야함.

위의 코딩식은 script 형태임

Demo in Gradio

In [6]:
def func (image):
  feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')
  model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')

  inputs = feature_extractor(images=image, return_tensors="pt")
  outputs = model(**inputs)
  logits = outputs.logits
  # model predicts one of the 1000 ImageNet classes
  predicted_class_idx = logits.argmax(-1).item()
  predicted_class = model.config.id2label[predicted_class_idx]
  return predicted_class  #이 함수는 이미지를 받아서 그것의 class를 text로 return한다

In [ ]:
gr.Interface(fn=func, inputs='image', outputs='text').launch()

In [ ]:
import os
url = "https://raw.githubusercontent.com/hsnam95/class2022Spring/main/tiger.jpg"
os.system("curl " + url + " > tiger.jpg")
url = "https://raw.githubusercontent.com/hsnam95/class2022Spring/main/dog.jpg"
os.system("curl " + url + " > dog.jpg")   #여기 구글 계정에 두 파일을 저장해라

In [ ]:
gr.Interface(fn=func, inputs='image', outputs='text', examples = ['tiger.jpg', 'dog.jpg']).launch()
#examples 적을때는 list화해서 적어줘야함.

### [Fill-Mask](https://huggingface.co/tasks/fill-mask)

e.g. https://huggingface.co/bert-base-uncased \
How to use

In [ ]:
!pip install transformers

In [14]:
from transformers import pipeline
unmasker = pipeline('fill-mask', model='bert-base-uncased')  #bert-base-uncased라는 모델을 들고 와서 그걸 unmasker라고 하자는 거임
unmasker("Hello I'm a [MASK] model.")   #결과를 보면 list로 되어있는데, list의 item 이 5개가 들어있음. dictionary가 list로 묶여있음

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'score': 0.10731077939271927,
  'sequence': "hello i'm a fashion model.",
  'token': 4827,
  'token_str': 'fashion'},
 {'score': 0.0877445638179779,
  'sequence': "hello i'm a role model.",
  'token': 2535,
  'token_str': 'role'},
 {'score': 0.05338413640856743,
  'sequence': "hello i'm a new model.",
  'token': 2047,
  'token_str': 'new'},
 {'score': 0.046672213822603226,
  'sequence': "hello i'm a super model.",
  'token': 3565,
  'token_str': 'super'},
 {'score': 0.027095887809991837,
  'sequence': "hello i'm a fine model.",
  'token': 2986,
  'token_str': 'fine'}]

Demo in Gradio

In [15]:
import pandas as pd
def func (text):
  unmasker = pipeline('fill-mask', model='bert-base-uncased')
  result = unmasker(text)
  df = pd.DataFrame(result)   #아까 list로 되어있던 result를 dataframe화하면 그냥 dataframe이 되는거임
  return df

In [ ]:
gr.Interface(fn=func, inputs='text', outputs='dataframe').launch()

In [17]:
examples = ["Hello I'm a [MASK] model.", "It is raining outside. I feel [MASK]."]

In [ ]:
gr.Interface(fn=func, inputs='text', outputs='dataframe', examples = examples).launch()

### [Token classification](https://huggingface.co/tasks/token-classification)

natural language understanding task 분야임
NER

e.g. https://huggingface.co/dslim/bert-base-NER \
How to use

In [ ]:
!pip install transformers

recognize four types of entities: location (LOC), organizations (ORG), person (PER) and Miscellaneous (MISC).

중요하다고 생각하는 entity만 뽑아내는거임
-> 위치, 조직(단체), 사람 .. 

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")
#image classification과 비슷하게 두개의 pretrained를 불러옴 (두개의 AI 모델을 불러오는거임) - 단어 수준으로 쪼개주는 모델 하나, 각각의 단어들이 어떤 class인지 recognition해주는 모델

nlp = pipeline("ner", model=model, tokenizer=tokenizer) 
example = "My name is Wolfgang and I live in Berlin"  

ner_results = nlp(example)   #nlp가 모델임
print(ner_results)   #결과를 보면, dataframe으로 해도 괜찮을거 같음 -> list로 되어있고, 내부적으로 2개의 dictionary로 되어있음

Demo in Gradio

In [24]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
def func (text):
  tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
  model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")
  nlp = pipeline("ner", model=model, tokenizer=tokenizer)
  result = nlp(text)
  df = pd.DataFrame(result)
  return df

In [ ]:
gr.Interface(fn=func, inputs='text', outputs='dataframe').launch()

In [27]:
examples = ["My name is Wolfgang and I live in Berlin", "I will visit Seoul to see Chris"]

In [ ]:
gr.Interface(fn=func, inputs='text', outputs='dataframe', examples = examples).launch()

### [Sentence similarity](https://huggingface.co/tasks/sentence-similarity)

두 주어진 문장이 있을때 얼마나 유사한가 보는거임

모든 model은 input을 raw data를 바로 받아들일 수 없음, 중간단계가, converting 단계가 필요함 -> feature라고 하기도 하고 vector라고 하기도 하고 embedding이라고 하기도 함. (capture semantic information and calculate ~)

e.g. https://huggingface.co/sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2 \
How to use

In [ ]:
!pip install sentence_transformers

In [ ]:
from sentence_transformers import SentenceTransformer, util
sentences = ["This is an example sentence", "it is one example writing"]  #list

model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
embeddings = model.encode(sentences)  #embeddings가 중요함.
print(embeddings)
#embeddings를 보면, [[     ], [      ]] 이런식으로 되어있음. 1번 list는 첫번째 text에 해당하고 2번 list는 두번째 text에 해당함.

In [ ]:
len(embeddings) #list가 2개 들어있으니까 2라고 나올거임

In [ ]:
len(embeddings[0])

In [ ]:
len(embeddings[1])   #384개의 숫자가 있구나를 이해하고 1번 list는 embedding for text1, 2번 list는 embedding for text2가 되는거임. 이 list안에 있는 숫자열이 feature vector인거임. 벡터는 숫자들이라고 했고, 데이터들이라고 했으니까 

[1,2]      [2,1]
이런 벡터가 있다고 할때, 이거를 좌표평면에 찍어볼 수 있음. 이 두개가 얼마나 비슷한 지는 원점과 그 점을 이은 선 사이의 각도로 이야기할 수 있음. 각도가 0도이면 유사도가 제일 높고 각도가 90도이면 유사도가 제일 낮다고 생각하면 됨.
cos(0) = 1 / cos(90) = 0이 됨.

similarity between 2 vectors(embedding)은 벡터들의 각도값의 cos(세타)라고 하면 됨.
-> 그걸 cos similarity라고 한다.

벡터가 만약에 숫자가 3개면, 3개의 각도, 4개면 4개의 각도 ....
차원이 아무리 늘어나도 두 벡터는 원점과 함께 삼각형을 만들게 되어있음. 각도가 항상 1개 나옴. 

-1<=cos(세타)=<1

In [33]:
cosine_scores = util.pytorch_cos_sim(embeddings[0], embeddings[1])  #cos similarity를 계산해줌
cosine_scores

tensor([[0.5726]])

Demo in Gradio

In [35]:
def func (text1, text2):
  from sentence_transformers import SentenceTransformer, util
  model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
  embeddings = model.encode([text1, text2])  #text 1과 2를 list화
  cosine_scores = util.pytorch_cos_sim(embeddings[0], embeddings[1])
  return cosine_scores

In [37]:
examples = [["This is an example sentence", "it is one example writing"], ["A frog is hopping near the pond", "I love Korean Food"]]

In [ ]:
gr.Interface(fn=func, inputs=['text', 'text'], outputs='number', examples = examples).launch()